# 1. Analysing Data

## Time Series Data

### Average Aggregate

The following query illustrates how to compute a simple average aggregate over all Time Series values in the table.

In [ ]:
%%sql
SELECT symbol, AVG(price)
FROM tick
GROUP BY symbol
ORDER BY symbol
LIMIT 5;

### Time Bucketing

Time bucketing can aggregate and group data for different time series by a fixed time interval. SingleStoreDB supports several functions:

- <strong>FIRST:</strong> The value associated with the minimum timestamp. The [documentation](https://docs.singlestore.com/db/latest/reference/sql-reference/time-series-functions/first.html) contains additional details and examples.
- <strong>LAST:</strong> The value associated with the maximum timestamp. The [documentation](https://docs.singlestore.com/db/latest/reference/sql-reference/time-series-functions/last/) contains additional details and examples.
- <strong>TIME_BUCKET:</strong> Normalizes time to the nearest bucket start time. The [documentation](https://docs.singlestore.com/db/latest/reference/sql-reference/time-series-functions/time-bucket/) contains additional details and examples.

For instance, we can use <strong>TIME_BUCKET</strong> to find the average time series value grouped by five-day intervals, as follows.

In [ ]:
%%sql
SELECT symbol, TIME_BUCKET("5d", ts), AVG(price)
FROM tick
WHERE symbol = "AAPL"
GROUP BY 1, 2
ORDER BY 1, 2
LIMIT 5;

#### Candlestick Charts

We can also combine these functions to create [candlestick charts](https://en.wikipedia.org/wiki/Candlestick_chart) that show the high, low, open, and close for a stock over time, bucketed by a window, as follows.

In [ ]:
%%sql
SELECT TIME_BUCKET("1d") AS ts,
     symbol,
     MIN(price) AS low,
     MAX(price) AS high,
     FIRST(price) AS open,
     LAST(price) AS close
FROM tick
WHERE symbol = "AAPL"
GROUP BY 2, 1
ORDER BY 2, 1;

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from sqlalchemy import *

conn = create_engine(connection_url)
num_days = input("How many days?")
symbol = input("Symbol?")

data = pd.read_sql("""
SELECT TIME_BUCKET(%s) AS day,
    symbol,
    MIN(price) AS low,
    MAX(price) AS high,
    FIRST(price) AS open,
    LAST(price) AS close
FROM tick
WHERE symbol = %s
GROUP BY 2, 1
ORDER BY 2, 1;
""", conn, params = (str(num_days) + "d", symbol.upper()))

fig = go.Figure(data = [go.Candlestick(
    x = data["day"],
    open = data["open"],
    high = data["high"],
    low = data["low"],
    close = data["close"],
    name = symbol,
  )])

fig.update_xaxes(type = "category")
fig.update_layout(xaxis_rangeslider_visible = False)
fig.show()

### Smoothing

We can smooth Time Series data using <strong>AVG</strong> as a windowed aggregate. Here is an example where we are looking at the price and the moving average of price over the last three ticks.

In [ ]:
%%sql
SELECT symbol, ts, price, AVG(price)
OVER (ORDER BY ts ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) AS smoothed_price
FROM tick
WHERE symbol = "AAPL"
LIMIT 5;

### AS OF

Finding a table row that is current <strong>AS OF</strong> a point in time is also a common Time Series requirement. This can be easily achieved using <strong>ORDER BY</strong> and <strong>LIMIT</strong>. Here is an example.

In [ ]:
%%sql
SELECT *
FROM tick
WHERE ts <= "2023-09-13 00:00:00"
AND symbol = "AAPL"
ORDER BY ts DESC
LIMIT 1;

---

## Sentiment Data

In [ ]:
%%sql
SELECT headline, positive, negative, neutral, symbol
FROM stock_sentiment
LIMIT 5;

---

# Common Code 2. and 3.

In [ ]:
!pip install langchain --quiet
!pip install openai --quiet

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

---

# 2. LangChain OnlinePDFLoader

In [ ]:
!pip install pdf2image --quiet
!pip install pdfminer.six --quiet
!pip install singlestoredb --quiet
!pip install tiktoken --quiet
!pip install unstructured --quiet

In [ ]:
from langchain.document_loaders import OnlinePDFLoader

loader = OnlinePDFLoader("<PDF document URL>")

data = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

print (f"You have {len(data)} document(s) in your data")
print (f"There are {len(data[0].page_content)} characters in your document")

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 20)
texts = text_splitter.split_documents(data)

print (f"You have {len(texts)} pages")

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

embedding = OpenAIEmbeddings()

In [ ]:
%%sql
DROP DATABASE IF EXISTS fintech;
CREATE DATABASE IF NOT EXISTS fintech;
USE fintech;
DROP TABLE IF EXISTS fintech_docs;

In [ ]:
import langchain.vectorstores.singlestoredb as s2
from langchain.vectorstores import SingleStoreDB
from langchain.vectorstores.utils import DistanceStrategy

s2.ORDERING_DIRECTIVE["DOT_PRODUCT"] = s2.ORDERING_DIRECTIVE[DistanceStrategy.DOT_PRODUCT]

docsearch = SingleStoreDB.from_documents(
    texts,
    embedding,
    table_name = "fintech.fintech_docs",
    distance_strategy = "DOT_PRODUCT",
)

In [ ]:
%%sql
USE fintech;
SELECT SUBSTRING(content, 1, 30) AS content,
       SUBSTRING(vector, 1, 20) AS vector
FROM fintech_docs
LIMIT 5;

In [ ]:
%%sql
USE fintech;
SELECT SUBSTRING(content, 1, 30) AS content,
       SUBSTRING(JSON_ARRAY_UNPACK(vector), 1, 70) AS vector
FROM fintech_docs
LIMIT 5;

In [ ]:
query_text = "What are the best investment opportunities in Blockchain?"

docs = docsearch.similarity_search(query_text)

print(docs[0].page_content)

In [ ]:
import openai

prompt = f"The user asked: {query_text}. The most similar text from the document is: {docs[0].page_content}"

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)

print(response['choices'][0]['message']['content'])

---

# 3. LangChain SQL Agent

In [ ]:
from langchain import OpenAI, SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent

In [ ]:
db = SQLDatabase.from_uri(connection_url)

llm = OpenAI(temperature = 0, verbose = False)

toolkit = SQLDatabaseToolkit(db = db, llm = llm)

agent_executor = create_sql_agent(
    llm = OpenAI(temperature = 0), toolkit = toolkit, verbose = False
)

In [ ]:
agent_executor.run(
"""
    From the tick table, which stock symbol saw the least volatility in share trading in the dataset?
"""
)

In [ ]:
query = input("Please enter your question:")
agent_executor.run(query)